<a href="https://colab.research.google.com/github/foxtrotmike/plotit/blob/master/plotit_regsurv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Sat May  1 04:21:16 2021

@author: u1876024
"""


# -*- coding: utf-8 -*-
"""
@author: Dr. Fayyaz Minhas
2D Scatter Plotter for Classification
"""
from numpy.random import randn #importing randn
import numpy as np #importing numpy
import matplotlib.pyplot as plt #importing plotting module
import itertools
import warnings

def plotit(X,Y=None,clf=None,  conts = None, ccolors = ('b','k','r'), colors = ('c','y'), markers = ('s','o'), hold = False, transform = None,extent = None,msize = None,vminmax=None,alpha=1.0,**kwargs):
    """
    A function for showing data scatter plot and classification boundary
    of a classifier for 2D data
        X: nxd  matrix of data points
        Y: (optional) n vector of class labels
        clf: (optional) classification/discriminant function handle
        conts: (optional) contours (if None, contours are drawn for each class boundary)
        ccolors: (optional) colors for contours   
        colors: (optional) colors for each class (sorted wrt class id)
            can be 'scaled' or 'random' or a list/tuple of color ids
        markers: (optional) markers for each class (sorted wrt class id)
        msize: markersize of each object (array equal to n, default: 30)
        vminmax: min and max value of color
        hold: Whether to hold the plot or not for overlay (default: False).
        transform: (optional) a function handle for transforming data before passing to clf
        kwargs: any keyword arguments to be passed to clf (if any)        
    """
    if clf is not None and X.shape[1]!=2:
        warnings.warn("Data Dimensionality is not 2. Unable to plot.")
        return
    if markers is None:
        markers = ('.',)
    if msize is None:
        msize=np.array([30]*X.shape[0])
    eps=1e-6
    d0,d1 = (0,1)
    if extent is None:
        minx, maxx = np.min(X[:,d0])-eps, np.max(X[:,d0])+eps
        miny, maxy = np.min(X[:,d1])-eps, np.max(X[:,d1])+eps
        extent = [minx,maxx,miny,maxy]
    else:
        [minx,maxx,miny,maxy] = extent
    if vminmax is not None:
        vmin,vmax = vminmax
    if Y is not None:
        classes = sorted(set(Y))
        if conts is None:
            conts = list(classes)    
        if vminmax is None:
            vmin,vmax = classes[0]-eps,classes[-1]+eps
    else:
        if vminmax is None:
            vmin,vmax=-2-eps,2+eps
        
        if conts is None:            
            conts = sorted([-1+eps,0,1-eps])
        
    if clf is not None:
        npts = 150
        x = np.linspace(minx,maxx,npts)
        y = np.linspace(miny,maxy,npts)
        t = np.array(list(itertools.product(x,y)))
        if transform is not None:
            t = transform(t)
        z = clf(t,**kwargs)
        
        z = np.reshape(z,(npts,npts)).T        
        
        
        plt.contour(x,y,z,conts,linewidths = [2],colors=ccolors,extent=extent, label='f(x)=0')
        #plt.imshow(np.flipud(z), extent = extent, cmap=plt.cm.Purples, vmin = -2, vmax = +2); plt.colorbar()
        plt.pcolormesh(x, y, z,cmap=plt.cm.Purples,vmin=vmin,vmax=vmax);plt.colorbar()
        plt.axis(extent)
    
    if Y is not None:        
        
        for i,y in enumerate(classes):
            if colors is None or colors=='scaled':
                cc = np.array([[i,i,i]])/float(len(classes))
            elif colors =='random':
                cc = np.array([[np.random.rand(),np.random.rand(),np.random.rand()]])
            else:
                cc = colors[i%len(colors)]
            mm = markers[i%len(markers)]
            
            sc = plt.scatter(X[Y==y,d0],X[Y==y,d1], marker = mm,c = cc, s = msize[Y==y],alpha=alpha)     
         
    else:
        sc = plt.scatter(X[:,d0],X[:,d1],marker = markers[0], c = 'k', s = msize,alpha=alpha)
    
    plt.xlabel('$x_1$')
    plt.ylabel('$x_2$')   
    if not hold:
        plt.grid()        
        plt.show()
    return extent,sc

def getExamples(n=100,d=2):
    """
    Generates n d-dimensional normally distributed examples of each class        
    The mean of the positive class is [1] and for the negative class it is [-1]
    """
    Xp = randn(n,d)#+1   #generate n examples of the positie class
    Xp=Xp+1
    Xn = randn(n,d)#-1   #generate n examples of the negative class
    Xn=Xn-1
    X = np.vstack((Xp,Xn))  #Stack the examples together to a single matrix
    Y = np.array([+1]*n+[-1]*n) #Associate Labels
    return (X,Y) 
       
        
if __name__ == '__main__':
    X,Y = getExamples()
    clf = lambda x: 2*np.sum(x,axis=1)-2.5 #dummy classifier
    plotit(X = X, Y = Y, clf = clf, conts =[-1,0,1], colors = 'random')
    """
    from plotit import *
    plt.figure()
    clf = svm_model.predict_risk #None#
    vminmax=(-1,4)
    X = X_train 
    Z = clf(X)
    vminmax = (np.percentile(Z,10),np.percentile(Z, 90))
    import umap
    mapper = umap.UMAP(random_state=42).fit(X)
    transform = mapper.inverse_transform
    X2 = mapper.transform(X)
    
    from sklearn.decomposition import PCA  
    pca = PCA(n_components=2).fit(X)     
    X2 = pca.transform(X) #let's use PCA to reduce dimensionality to 2
    transform = pca.inverse_transform
    
    extent,sc = plotit(X=X2,Y=np.array(E_train),transform=transform,clf=clf,markers=['o','o'],msize=10+T_train*20,vminmax=vminmax,colors=('g','r'),conts=[],alpha=0.5)
    kw = dict(prop="sizes", num=6,  fmt="{x:.0f}", func=lambda s: (s-10)/20)
    #plt.legend(*sc.legend_elements("sizes", num=6))
    plt.legend(*sc.legend_elements(**kw))
    """